In [173]:
# import libraries
import pandas as pd
import json
from arango import ArangoClient
import re
import gdown
from datetime import datetime

In [34]:
# Arango Client Connection
client = ArangoClient(hosts= "http://localhost:8529")

# database connection
db = client.db('Goodreads_Book_Reviews', username = 'root', password = 'bolajimartinArango')

## Creating First Collection - Top_100_Books

Process:
- Read File from Google Drive and Change to JSON if file type is not Json
- Obtain Keys in the File
- Create collection (Top_100_Books)in the Goodreads_Book_Reviews Database
- Define the Schemas for the keys in the document and load document into the collection - Top_100_Books

In [301]:
# read file to be loaded into Arango from google drive into Python
url = "https://drive.google.com/file/d/1FuHlnpkGxleewKeCesZMAQYmhs-lB_Jb/view?usp=share_link"
file_path = "https://drive.google.com/uc?id=" + url.split('/')[-2]
top100books = pd.read_csv(file_path)

### data cleaning and transformation

In [307]:
# view data in csv
top100books.sample(2)

,Book Title,Book Link,Review Page Link,Unique ID
10,Gone with the Wind,https://www.goodreads.com/book/show/18405.Gone...,https://www.goodreads.com/book/show/18405/revi...,18405
89,The Berlin Stories,https://www.goodreads.com/book/show/16810.The_...,https://www.goodreads.com/book/show/16810/revi...,16810


In [315]:
# obtain the keys in the json file
columns_top100books = [i.lower().replace(' ', '_') for i in list(top100books.keys())]

for i in range(len(columns_top100books)):
    top100books = top100books.rename(columns = {top100books.columns[i]: columns_top100books[i]})

top100books.columns

Index(['book_title', 'book_link', 'review_page_link', 'unique_id'], dtype='object')

In [316]:
# transform into json format to load into Arango
top_100books_json = json.loads(top100books.to_json(orient='records'))

### data loading

In [317]:
# define schemas for the keys in the json file
top100books_schema = {
    'rule': {
        'type': 'object',
        'properties': {
            'book_title': {'type': 'string'},
            'book_link': {'type': 'string'},
            'review_page_link': {'type': 'string'},
            'unique_id' : {'type': 'integer'}
            
        },
        'required': ['book_title', 'book_link', 'review_page_link', 'unique_id']
    },
    'level': 'moderate',
    'message': 'Schema Validation Failed.'
}

# create collection in the database
top_100books_collection = db.create_collection('Top_100_Books', schema = top100books_schema)

# import json file into collection - top_100books_collection
top_100books_collection.import_bulk(top_100books_json)

{'error': False,
 'created': 100,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

## Create Second Collection - Book Reviews

- Read File from Google Drive and Change to JSON if file type is not Json
- Obtain Keys in the File
- Create collection (Book_Reviews)in the Goodreads_Book_Reviews Database
- Define the Schemas for the keys in the document and load document into the collection - Book_Reviews

In [318]:
url_bookreviews = "https://drive.google.com/file/d/1SCrhzFD_pouvKb1njckacKsBWdSPMPjB/view?usp=share_link"
fileurl_bookreviews = "https://drive.google.com/uc?id=" + url_book1.split('/')[-2]

# download file from google drive for easy loading into Python due to large file size preventing google drive from scanning
bookReviews = gdown.download(fileurl_bookreviews, 'BookReviews.csv', quiet=True)

# load downloaded data into pandas
book_reviews = pd.read_csv(bookReviews)

### data cleaning and transformation

In [319]:
book_reviews.head(2)

,User Href,Title,Rating,Date,Likes,Comments,Review,Display Name,User ID,Language
0,https://www.goodreads.com/user/show/45198798-l...,To Kill a Mockingbird,0.2,03-24-2022,964,88,/// gentle reminder that this is not the time ...,leynes,45198798,en
1,https://www.goodreads.com/user/show/1413439-st...,To Kill a Mockingbird,1.0,05-24-2011,1445,104,6.0 stars. I know I am risking a serious “FILM...,stephen,1413439,en


In [320]:
columns_book_reviews = [i.lower().replace(' ', '_') for i in list(book_reviews.keys())]

for i in range(len(columns_book_reviews)):
    book_reviews = book_reviews.rename(columns = {book_reviews.columns[i]: columns_book_reviews[i]})

book_reviews.columns

Index(['user_href', 'title', 'rating', 'date', 'likes', 'comments', 'review',
       'display_name', 'user_id', 'language'],
      dtype='object')

In [321]:
def datacleaning(dataframe):
    # multiply ratings by 5 to get the actual ratings
    dataframe['rating'] = dataframe['rating'].apply(lambda x: int(x * 5))
    
    return dataframe

# update dataframe
book_reviews = datacleaning(book_reviews)

# transform to json
book_reviews_json = json.loads(book_reviews.to_json(orient = 'records'))

### data loading

In [325]:
# define schemas for the keys in the json file
book_reviews_schema = {
    'rule': {
        'type': 'object',
        'properties': {
            'user_href': {'type': ['string', 'null']},
            'title': {'type': ['string', 'null']},
            'rating': {'type': ['integer', 'null']},
            'date' : {'type': ['string', 'null']},
            'likes' : {'type': ['integer', 'null']},
            'comments' : {'type': ['integer', 'null']},
            'review' : {'type': ['string', 'null']},
            'display_name': {'type': ['string', 'null']},
            'user_id' : {'type': ['integer', 'null']},
            'language' : {'type': ['string', 'null']}  
        },
        'required': ['user_href', 'title', 'rating', 'date', 'likes', 'comments', 'review', 'display_name', 'user_id', 'language']
    },
    'level': 'moderate',
    'message': 'Schema Validation Failed.'
}

# create collection in the database
book_reviews_collection = db.create_collection('Book_Reviews', schema = book_reviews_schema)

# import json file into collection - Book_Reviews
book_reviews_collection.import_bulk(book_reviews_json)

{'error': False,
 'created': 100359,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

## Create Edge Collection Between Top 100 Books and Book Reviews

query for insertion in to edge document:
- for each book in the top_100_books collection, filter its reviews in the Book_Reviews collection by matching the book_title field in Top_100_Books to the title field in the Book_Reviews collection. Afterwhich define the connecting edges between both collections using the '._id' field

In [341]:
query = '''
FOR book IN Top_100_Books
    FOR reviews IN Book_Reviews
        FILTER reviews.title == book.book_title
        INSERT {
            _from: book._id,
            _to: reviews._id
        } INTO reviews_per_book
'''

# create edge collection
db.create_collection('reviews_per_book', edge=True)
cursor = db.aql.execute(query)